# Clapton

Read the CAFQA notebook first to understand the basic objects!

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
from clapton.depolarization import GateGeneralDepolarizationModel
from clapton.evaluation import transform_paulis
import numpy as np

In [2]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))

In [3]:
# define empty VQE Clifford circuit that captures noise but essentially acts as
# identity
# here we use the circular_ansatz template
# we fix 2q gates
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [4]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)
vqe_pcirc.add_depolarization_model(nm)

In [5]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [6]:
# we need a circuit that parametrizes the Clapton transformation
# also use circular ansatz (could be different) but allow optimization over 2q gates
trans_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=False)

In [7]:
# we can perform Clapton by using the main optimization function "claptonize"
# vqe_pcirc will stay fixed, optimization happens over trans_pcirc
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    trans_pcirc,        # specify transformation circuit
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id None with 1 procs



d:\research-impl\clapton\clapton\venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


[0, array([-1.92982046, -0.93421786, -0.9956026 ,  0.        ]), array([1, 2, 3, 3, 3, 2, 0, 0, 0, 3, 3, 2, 3, 1, 1], dtype=object)]
[1, array([-3.30118125, -1.57811873, -1.72306252,  0.        ]), array([0, 1, 3, 3, 1, 0, 0, 0, 0, 3, 3, 0, 3, 2, 3], dtype=object)]
[2, array([-3.32893646, -1.58126029, -1.74767617,  0.        ]), array([1, 3, 0, 3, 0, 2, 0, 2, 0, 1, 1, 0, 2, 3, 0], dtype=object)]
[3, array([-3.31508118, -1.59201866, -1.72306252,  0.        ]), array([0, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 0, 3, 2, 3], dtype=object)]
[4, array([-3.29287221, -1.56980969, -1.72306252,  0.        ]), array([0, 3, 3, 1, 1, 2, 0, 0, 0, 1, 2, 0, 3, 2, 3], dtype=object)]
[5, array([-3.32550891, -1.60244639, -1.72306252,  0.        ]), array([0, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 0, 3, 2, 3], dtype=object)]
[6, array([-3.29630188, -1.57323936, -1.72306252,  0.        ]), array([1, 2, 1, 3, 3, 2, 0, 0, 3, 0, 2, 1, 2, 3, 3], dtype=object)]
[7, array([-3.3243508 , -1.60128828, -1.72306252,  0.        ]), arra

In [8]:
# the best parameters are
ks_best

[3, 3, 1, 1, 1, 0, 0, 1, 1, 1, 2, 3, 3, 2, 3]

In [9]:
# with noisy/noiseless energies
energy_noisy, energy_noiseless

(np.float64(-1.601676640736366), np.float64(-1.747676167007962))

In [10]:
# the transformation circuit is
trans_pcirc.assign(ks_best)
trans_pcirc.stim_circuit().diagram()

q0: -SQRT_Y_DAG-S-@-SQRT_Y-S_DAG------------
                  |
q1: -SQRT_Y_DAG-S-X-@------Y----------Z-----
                    |
q2: -SQRT_Y-----I---X------SQRT_Y_DAG-S_DAG-

In [11]:
# the transformed Hamiltonian is given by
paulis_new, signs = transform_paulis(trans_pcirc.stim_circuit(), paulis, replace_I=True)
coeffs_new = np.multiply(coeffs, signs)
paulis_new, coeffs_new

(['ZIZ', 'XZX', 'IZZ', 'IYY', 'ZZI', 'XXZ'],
 array([-0.16311038,  0.05030232, -0.72745992,  0.9956026 , -0.85710587,
        -0.3331151 ]))